### **Laboratorio 1**

### **Carga de Datos de Importacion Hidrocarburos**

In [10]:
import pandas as pd

df_import_2024 = pd.read_excel("IMPORTACION-HIDROCARBUROS-VOLUMEN-2024-12.xlsx", skiprows=6)
df_import_2025 = pd.read_excel("IMPORTACION-HIDROCARBUROS-VOLUMEN-2025-05.xlsx", skiprows=6)

df_total = pd.concat([df_import_2024, df_import_2025], ignore_index=True)






In [11]:
df_total.head()  # Mostrar las primeras filas para verificar la carga de datos

,Fecha,Aceites lubricantes,Asfalto,Bunker,Ceras,Combustible turbo jet,Diesel bajo azufre,Diesel ultra bajo azufre,Gas licuado de petróleo,Gasolina de aviación,...,Diesel alto azufre,MTBE,Orimulsión,Petróleo reconstituido,Total importación,Aceites Lubricantes,Asfalto,Bunker C o Fuel Oil,Gas Licuado de Petróleo,Grasas Lubricantes
0,2001-01-01 00:00:00,0.0,27748.99,214581.84,0.0,0.0,0.0,0.0,194065.738095,820.0,...,566101.99,8402.0,0.0,715344.0,2.312639e+06,NaN,NaN,NaN,NaN,NaN
1,2001-02-01 00:00:00,0.0,7503.57,294609.00,0.0,0.0,0.0,0.0,170703.380952,3054.0,...,489525.80,0.0,0.0,370166.0,1.769209e+06,NaN,NaN,NaN,NaN,NaN
2,2001-03-01 00:00:00,0.0,26304.32,315263.80,0.0,0.0,0.0,0.0,161837.371429,677.0,...,575559.68,0.0,0.0,360530.0,1.945770e+06,NaN,NaN,NaN,NaN,NaN
3,2001-04-01 00:00:00,0.0,7885.89,205653.00,0.0,0.0,0.0,0.0,163048.642857,3399.0,...,437745.42,8184.0,0.0,359527.0,1.623638e+06,NaN,NaN,NaN,NaN,NaN
4,2001-05-01 00:00:00,0.0,8443.16,278371.30,0.0,0.0,0.0,0.0,171518.861905,585.0,...,552609.13,12680.0,0.0,723346.0,2.262727e+06,NaN,NaN,NaN,NaN,NaN
